# Step 1: Generate Answer Sheets
Create personalized answer sheets by merging the name list into the template, exporting per-student PDFs, and combining them for printing.

In [1]:
from grading_utils import setup_paths

prefix = "VTC Test"
paths = setup_paths(prefix, "sample")

name_list = paths["name_list_file"]
answer_sheet = f"../sample/{prefix} Answer Sheet.docx"

In [2]:
import pandas as pd

df = pd.read_excel(name_list)
df.head()


,NAME,ID,CLASS
0,Peter,123456789,A
1,Mary,987654321,B
2,John,234567890,C
3,Susan,345678912,D


In [3]:
# Validate that Student IDs are unique
duplicate_ids = df[df.duplicated(subset=['ID'], keep=False)].sort_values('ID')

if not duplicate_ids.empty:
    print("⚠️ ERROR: Duplicate Student IDs detected!")
    print("\nDuplicate records:")
    print(duplicate_ids[['ID', 'NAME', 'CLASS']])
    raise ValueError("Cannot proceed: Student IDs must be unique")
else:
    print("✓ Student ID validation passed - all IDs are unique")
    print(f"Total students: {len(df)}")

✓ Student ID validation passed - all IDs are unique
Total students: 4


In [4]:
from docx import Document
from pypdf import PdfWriter, PdfReader
import subprocess
import os

# Create individual PDFs for each student
individual_pdfs = []

for index, row in df.iterrows():
    # Load the template
    doc = Document(answer_sheet)
    
    # Replace placeholders in all paragraphs (preserving formatting)
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if 'Name:' in run.text:
                run.text = run.text.replace('Name:', f"Name: {row['NAME']}")
            if 'Student ID:' in run.text:
                run.text = run.text.replace('Student ID:', f"Student ID: {row['ID']}")
            if 'Class:' in run.text:
                run.text = run.text.replace('Class:', f"Class: {row['CLASS']}")
    
    # Replace placeholders in tables
    for table in doc.tables:
        for row_table in table.rows:
            for cell in row_table.cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        if 'Name:' in run.text:
                            run.text = run.text.replace('Name:', f"Name: {row['NAME']}")
                        if 'Student ID:' in run.text:
                            run.text = run.text.replace('Student ID:', f"Student ID: {row['ID']}")
                        if 'Class:' in run.text:
                            run.text = run.text.replace('Class:', f"Class: {row['CLASS']}")
    
    # Save the modified docx using ID as filename (unique identifier)
    docx_filename = f"../data/{prefix} Answer Sheet - {row['ID']}.docx"
    doc.save(docx_filename)
    
    # Convert to PDF using LibreOffice
    pdf_filename = f"../data/{prefix} Answer Sheet - {row['ID']}.pdf"
    result = subprocess.run([
        'libreoffice', '--headless', '--convert-to', 'pdf', 
        '--outdir', '../data', docx_filename
    ], capture_output=True, text=True)
    
    individual_pdfs.append(pdf_filename)
    print(f"Created PDF for {row['NAME']} (ID: {row['ID']}): {pdf_filename}")

# Merge all PDFs
if individual_pdfs:
    writer = PdfWriter()
    
    for pdf_file in individual_pdfs:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            writer.add_page(page)
    
    # Write merged PDF
    output_file = f"../data/{prefix} Answer Sheets Combined.pdf"
    with open(output_file, 'wb') as output:
        writer.write(output)
    
    print(f"\nCombined PDF saved to: {output_file}")
    print(f"Total students processed: {len(df)}")
    
    # Clean up intermediate files
    print("\nCleaning up intermediate files...")
    for index, row in df.iterrows():
        docx_file = f"../data/{prefix} Answer Sheet - {row['ID']}.docx"
        pdf_file = f"../data/{prefix} Answer Sheet - {row['ID']}.pdf"
        if os.path.exists(docx_file):
            os.remove(docx_file)
        if os.path.exists(pdf_file):
            os.remove(pdf_file)
    print("Cleanup complete!")

Created PDF for Peter (ID: 123456789): ../data/VTC Test Answer Sheet - 123456789.pdf
Created PDF for Mary (ID: 987654321): ../data/VTC Test Answer Sheet - 987654321.pdf
Created PDF for John (ID: 234567890): ../data/VTC Test Answer Sheet - 234567890.pdf
Created PDF for Susan (ID: 345678912): ../data/VTC Test Answer Sheet - 345678912.pdf

Combined PDF saved to: ../data/VTC Test Answer Sheets Combined.pdf
Total students processed: 4

Cleaning up intermediate files...
Cleanup complete!
